## 歌曲序列建模
### 从word2vec到song2vec
把歌曲的id序列取出来，类比于分完词后的句子，送到word2vec中去学习

In [8]:
#coding: utf-8
import multiprocessing
import gensim
import sys
from random import shuffle

def parse_playlist_get_sequence(in_line, playlist_sequence):
    song_sequence = []
    contents = in_line.strip().split("\t")
    # 解析歌单序列
    for song in contents[1:]:
        try:
            song_id, song_name, artist, popularity = song.split("::")
            song_sequence.append(song_id)
        except:
            print ("song format error")
            print (song+"\n")
    for i in range(len(song_sequence)):
        shuffle(song_sequence)
        playlist_sequence.append(song_sequence)


def train_song2vec(in_file, out_file):
    #所有歌单序列
    playlist_sequence = []
    #遍历所有歌单
    for line in open(in_file, encoding='utf-8'):
        parse_playlist_get_sequence(line, playlist_sequence)
    #使用word2vec训练
    cores = multiprocessing.cpu_count()
    print ("using all "+str(cores)+" cores")
    print ("Training word2vec model...")
    model = gensim.models.Word2Vec(sentences=playlist_sequence, size=150, min_count=3, window=7, workers=cores)
    print ("Saving model...")
    model.save(out_file)

In [9]:
song_sequence_file = "./ori_data/popular.playlist"
model_file = "./model/song2vec.model"
train_song2vec(song_sequence_file, model_file)

song format error
1870957::彩云国物语 セカンドシリーズ::君を想う::梁邦彦::80.0

song format error
4965888::桃华月惮::龙皇-リュウオウ-::多田彰文::25.0

song format error
456177::true tears::一阵の风::菊地創::95.0

song format error
22642373::

song format error
 FAIRY TAIL メインテーマ -Slow ver.-::高梨康治::95.0

song format error
31563610::

song format error
苍之礼赞::花之祭P::60.0

song format error
4954593::リズム天国全曲集::恋の実験室::V.A.::55.0

song format error
4954596::リズム天国全曲集::シンクロ::V.A.::60.0

song format error
31654811::

song format error
American Cowboys::Tim Wynn::65.0

song format error
19169096::

song format error
 Time to Say Goodbye (Con te partirò)::Sarah Brightman::100.0

song format error
31563610::

song format error
苍之礼赞::花之祭P::60.0

song format error
31563610::

song format error
苍之礼赞::花之祭P::60.0

song format error
31563610::

song format error
苍之礼赞::花之祭P::60.0

song format error
19169096::

song format error
 Time to Say Goodbye (Con te partirò)::Sarah Brightman::100.0

song format error
376653::野弧禅狂叱(宿香之战)

song format error
:

### 预测的过程，实际上就是对某首歌曲，查找“最近”的歌曲（向量距离最近的歌曲）

In [10]:
import pickle
song_dic = pickle.load(open("./pro_data/popular_song.pkl","rb"))
model_str = "./model/song2vec.model"
model = gensim.models.Word2Vec.load(model_str)

In [12]:
for song in list(song_dic.keys())[:10]:
    print (song, song_dic[song])

315958 那件疯狂的小事叫爱情	袁泉
28138980 为你我受冷风吹	孙露
247526 彗星的眼泪	金莎
5280395 慨古吟(琴歌)	张铜霞
31140395 一首简单的歌	本兮
27532150 Smoke Fly ft. JBo Escobar & Khaki	Al Rocco
440767373 メドゥーサ（美杜莎）	月蝕原创音乐
16323636 The Prayer	Andrea Bocelli
281436 夜曲	彭芳
5270404 渴望(二胡)	群星


In [14]:
song_id_list = list(song_dic.keys())[1000:1500:50]
for song_id in song_id_list:
    result_song_list = model.most_similar(song_id)

    print (song_id, song_dic[song_id])
    print ("\n相似歌曲 和 相似度 分别为:")
    for song in result_song_list:
        print ("\t", song_dic[song[0]], song[1])
    print ("\n")

D:\Anaconda\install\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


368971 Ambulance of love	脑浊

相似歌曲 和 相似度 分别为:
	 新世界	呼吸 0.8102102279663086
	 上苍保佑吃完了饭的人民	张楚 0.8082322478294373
	 呀呀	图腾 0.7943791151046753
	 昨日我从清晨开始等待	钟立风 0.774426281452179
	 生命(Live) - live	声音玩具 0.7557182312011719
	 两天	许巍 0.7442638874053955
	 永远在一起	飘乐队 0.7283462285995483
	 今夜	许巍 0.7184534072875977
	 祖先的阴影	超载 0.7170863747596741
	 我们走过的路	天空 0.7092562913894653


33599059 八秒之语	洛天依

相似歌曲 和 相似度 分别为:
	 乡村DISCO	VOCALOID 0.6794760227203369
	 春雨	乐正绫 0.649375319480896
	 远恋	阿良良木健 0.6485384702682495
	 食之歌 VOCALOID Ver.	泛音堂 0.6450830698013306
	 小幸运（Cover：田馥甄）	星魂梦 0.6226800680160522
	 出格	阿妍 0.6186865568161011
	 双向监禁	洛天依 0.6172651052474976
	 山海默示录（洛天依版）	小旭PRO 0.6170870661735535
	 甄姬	VOCALOID 0.613639771938324
	 全世界都死了	海鲜面 0.6097506284713745


408332846 知足	苏运莹

相似歌曲 和 相似度 分别为:
	 垃圾车（cover 五月天）	李昂星 0.714142918586731
	 Happy Birth Day	香蕉 0.7014893293380737
	 拥抱（Cover 五月天)	橙大蕾蕾 0.6977276802062988
	 爱情的模样	小平 0.6919869780540466
	 听不到(Live)	梁静茹 0.6528257727622986
	 我就是这样的	黄贯中 0.6462737917900085
	 拥抱（Cover 五月天